In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# December 16 games
# line 6.5
bcstpete = MoneyLine(event="bcstpete", bet_amount=100, odds=240)
maccabi = MoneyLine(event="maccabi", bet_amount=100, odds=-305)

# line 5.5
zalgriris = MoneyLine(event="zalgriris", bet_amount=100, odds=210)
anadolu = MoneyLine(event="anadolu", bet_amount=100, odds=-265)

# line 4.5
fenerbahce = MoneyLine(event="fenerbahce", bet_amount=100, odds=155)
panathinkaikos = MoneyLine(event="panathinkaikos", bet_amount=100, odds=-195)

# line 10.5
axarmani = MoneyLine(event="axarmani", bet_amount=100, odds=475)
realmadrid = MoneyLine(event="realmadrid", bet_amount=100, odds=-715)


binaries = [
    [bcstpete, maccabi],
    [zalgriris, anadolu],
    [fenerbahce, panathinkaikos],
    [axarmani, realmadrid],
]

results = [
    [0, 1], #[bcstpete, maccabi],
    [0, 1], #[zalgriris, anadolu],
    [0, 1], #[fenerbahce, panathinkaikos],
    [0, 1], #[axarmani, realmadrid],
]

def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

1

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=1,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.49082559846375295
     jac: array([-2.484625  , -1.11814103, -0.61771227, -0.01057088, -0.43710041,
        0.09657945,  0.29202173,  0.5291406 ])
 message: 'Inequality constraints incompatible'
    nfev: 30
     nit: 3
    njev: 3
  status: 4
 success: False
       x: array([0.08072334, 0.13606672, 0.18168281, 0.2133352 , 0.20669162,
       0.19539903, 0.14570254, 0.01      ])
slsqp_solver: 
                               event    index[]     result  event_status  \
7     maccabi_anadolu_panathinkaikos  [1, 3, 5]  [1, 1, 1]          True   
0      bcstpete_zalgriris_fenerbahce  [0, 2, 4]  [0, 0, 0]         False   
1  bcstpete_zalgriris_panathinkaikos  [0, 2, 5]  [0, 0, 1]         False   
2        bcstpete_anadolu_fenerbahce  [0, 3, 4]  [0, 1, 0]         False   
4       maccabi_zalgriris_fenerbahce  [1, 2, 4]  [1, 0, 0]         False   
3    bcstpete_anadolu_panathinkaikos  [0, 3, 5]  [0, 1, 1]         False   
5   maccabi_zalgriris_panathinkaikos  [1, 2, 5]  [1, 0, 1] 

In [8]:
df.describe()

,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000
mean,0.147500,10.750408,1.514225,0.345000
std,0.070862,7.775558,0.820545,0.819895
min,0.010000,2.766875,0.027700,-1.140000
25%,0.125000,5.836480,1.082475,-0.085000
50%,0.165000,8.790685,1.840500,0.670000
75%,0.202500,12.942556,2.169600,1.000000
max,0.210000,26.877000,2.169600,1.000000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,2.000000,2.000000,2.000000,2.000000
mean,0.080000,3.715351,0.353600,-0.815000
std,0.098995,1.341347,0.460892,0.459619
min,0.010000,2.766875,0.027700,-1.140000
25%,0.045000,3.241113,0.190650,-0.977500
50%,0.080000,3.715351,0.353600,-0.815000
75%,0.115000,4.189588,0.516550,-0.652500
max,0.150000,4.663826,0.679500,-0.490000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,6.000000,6.000000,6.000000,6.000000
mean,0.170000,13.095427,1.901100,0.731667
std,0.051381,7.608380,0.426263,0.425696
min,0.080000,6.227364,1.216800,0.050000
25%,0.150000,7.937626,1.675950,0.505000
50%,0.190000,11.219251,2.169600,1.000000
75%,0.207500,14.944271,2.169600,1.000000
max,0.210000,26.877000,2.169600,1.000000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
7,maccabi_anadolu_panathinkaikos,"[1, 3, 5]","[1, 1, 1]",True,0.01,2.766875,0.0277,-1.14
6,maccabi_anadolu_fenerbahce,"[1, 3, 4]","[1, 1, 0]",False,0.15,4.663826,0.6795,-0.49


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
0,bcstpete_zalgriris_fenerbahce,"[0, 2, 4]","[0, 0, 0]",False,0.08,26.877000,2.1696,1.00
1,bcstpete_zalgriris_panathinkaikos,"[0, 2, 5]","[0, 0, 1]",False,0.14,15.945128,2.1696,1.00
2,bcstpete_anadolu_fenerbahce,"[0, 3, 4]","[0, 1, 0]",False,0.18,11.941698,2.1696,1.00
4,maccabi_zalgriris_fenerbahce,"[1, 2, 4]","[1, 0, 0]",False,0.21,10.496803,2.1696,1.00
3,bcstpete_anadolu_panathinkaikos,"[0, 3, 5]","[0, 1, 1]",False,0.21,7.084567,1.5114,0.34
5,maccabi_zalgriris_panathinkaikos,"[1, 2, 5]","[1, 0, 1]",False,0.20,6.227364,1.2168,0.05


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,6,6,6,6,6,6,6


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,1,1,1,1,1,1,1
True,1,1,1,1,1,1,1


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

def color_positive_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'green' if val == True else 'black'
    return 'color: %s' % color

In [17]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,maccabi_anadolu_panathinkaikos,"['1', '3', '5']","[1, 1, 1]",True,0.01,2.76688,0.0277,-1.14
1,maccabi_anadolu_fenerbahce,"['1', '3', '4']","[1, 1, 0]",False,0.15,4.66383,0.6795,-0.49


In [18]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,bcstpete_zalgriris_fenerbahce,"['0', '2', '4']","[0, 0, 0]",False,0.08,26.877,2.1696,1
1,bcstpete_zalgriris_panathinkaikos,"['0', '2', '5']","[0, 0, 1]",False,0.14,15.9451,2.1696,1
2,bcstpete_anadolu_fenerbahce,"['0', '3', '4']","[0, 1, 0]",False,0.18,11.9417,2.1696,1
3,maccabi_zalgriris_fenerbahce,"['1', '2', '4']","[1, 0, 0]",False,0.21,10.4968,2.1696,1
4,bcstpete_anadolu_panathinkaikos,"['0', '3', '5']","[0, 1, 1]",False,0.21,7.08457,1.5114,0.34
5,maccabi_zalgriris_panathinkaikos,"['1', '2', '5']","[1, 0, 1]",False,0.2,6.22736,1.2168,0.05
